In [1]:
import math
import pandas as pd
import numpy as np
from operator import itemgetter

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

sns.set()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

INFILE = "C:\\Users\\Owner\\Documents\\Northwestern\\Intro_MachineLearning\\Assignment1\\DataSet\\HMEQ_Loss.csv"

TARGET_F = "TARGET_BAD_FLAG"
TARGET_A = "TARGET_LOSS_AMT"

df = pd.read_csv( INFILE )
df_org = df.copy()


dt = df.dtypes

In [2]:
print( df.head().T )
print('-----------------------')
print( df.dtypes )

                       0        1        2     3        4
TARGET_BAD_FLAG        1        1        1     1        0
TARGET_LOSS_AMT      641     1109      767  1425      NaN
LOAN                1100     1300     1500  1500     1700
MORTDUE            25860    70053    13500   NaN    97800
VALUE              39025    68400    16700   NaN   112000
REASON           HomeImp  HomeImp  HomeImp   NaN  HomeImp
JOB                Other    Other    Other   NaN   Office
YOJ                 10.5        7        4   NaN        3
DEROG                  0        0        0   NaN        0
DELINQ                 0        2        0   NaN        0
CLAGE            94.3667  121.833  149.467   NaN  93.3333
NINQ                   1        0        1   NaN        0
CLNO                   9       14       10   NaN       14
DEBTINC              NaN      NaN      NaN   NaN      NaN
-----------------------
TARGET_BAD_FLAG      int64
TARGET_LOSS_AMT    float64
LOAN                 int64
MORTDUE            float6

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
TARGET_BAD_FLAG,5960.0,0.199497,0.399656,0.000000,0.000000,0.000000,0.000000,1.000000
TARGET_LOSS_AMT,1189.0,13414.576955,10839.455965,224.000000,5639.000000,11003.000000,17634.000000,78987.000000
LOAN,5960.0,18607.969799,11207.480417,1100.000000,11100.000000,16300.000000,23300.000000,89900.000000
MORTDUE,5442.0,73760.817200,44457.609458,2063.000000,46276.000000,65019.000000,91488.000000,399550.000000
VALUE,5848.0,101776.048741,57385.775334,8000.000000,66075.500000,89235.500000,119824.250000,855909.000000
YOJ,5445.0,8.922268,7.573982,0.000000,3.000000,7.000000,13.000000,41.000000
DEROG,5252.0,0.254570,0.846047,0.000000,0.000000,0.000000,0.000000,10.000000
DELINQ,5380.0,0.449442,1.127266,0.000000,0.000000,0.000000,0.000000,15.000000
CLAGE,5652.0,179.766275,85.810092,0.000000,115.116702,173.466667,231.562278,1168.233561
NINQ,5450.0,1.186055,1.728675,0.000000,0.000000,1.000000,2.000000,17.000000


In [4]:
x = df.describe().T

objList = []
numList = []
for i in dt.index :
    #print(" here is i .....", i , " ..... and here is the type", dt[i] )
    if i in ( [ TARGET_F, TARGET_A ] ) : continue
    if dt[i] in (["object"]) : objList.append( i )
    if dt[i] in (["float64","int64"]) : numList.append( i )

print(" OBJECTS ")
print(" ------- ")
for i in objList :
    print(i)
##
print("  ")
print(" NUMBERS ")
print(" ------- ")
for i in numList :
    print(i)

 OBJECTS 
 ------- 
REASON
JOB
  
 NUMBERS 
 ------- 
LOAN
MORTDUE
VALUE
YOJ
DEROG
DELINQ
CLAGE
NINQ
CLNO
DEBTINC


In [5]:
"""
FILL IN MISSING WITH THE CATEGORY "MISSING"
"""
for i in objList :
    if df[i].isna().sum() == 0 : continue
    NAME = "IMP_"+i
    df[NAME] = df[i]
    df[NAME] = df[NAME].fillna("MISSING")
    g = df.groupby( NAME )
    df = df.drop( i, axis=1 )


dt = df.dtypes
objList = []
for i in dt.index :
    print(" here is i .....", i , " ..... and here is the type", dt[i] )
    if i in ( [ TARGET_F, TARGET_A ] ) : continue
    if dt[i] in (["object"]) : objList.append( i )

 here is i ..... TARGET_BAD_FLAG  ..... and here is the type int64
 here is i ..... TARGET_LOSS_AMT  ..... and here is the type float64
 here is i ..... LOAN  ..... and here is the type int64
 here is i ..... MORTDUE  ..... and here is the type float64
 here is i ..... VALUE  ..... and here is the type float64
 here is i ..... YOJ  ..... and here is the type float64
 here is i ..... DEROG  ..... and here is the type float64
 here is i ..... DELINQ  ..... and here is the type float64
 here is i ..... CLAGE  ..... and here is the type float64
 here is i ..... NINQ  ..... and here is the type float64
 here is i ..... CLNO  ..... and here is the type float64
 here is i ..... DEBTINC  ..... and here is the type float64
 here is i ..... IMP_REASON  ..... and here is the type object
 here is i ..... IMP_JOB  ..... and here is the type object


In [6]:
df.head().T

,0,1,2,3,4
TARGET_BAD_FLAG,1,1,1,1,0
TARGET_LOSS_AMT,641,1109,767,1425,NaN
LOAN,1100,1300,1500,1500,1700
MORTDUE,25860,70053,13500,NaN,97800
VALUE,39025,68400,16700,NaN,112000
YOJ,10.5,7,4,NaN,3
DEROG,0,0,0,NaN,0
DELINQ,0,2,0,NaN,0
CLAGE,94.3667,121.833,149.467,NaN,93.3333
NINQ,1,0,1,NaN,0


In [7]:
dt = df.dtypes
objList = []
for i in dt.index :
    print(" here is i .....", i , " ..... and here is the type", dt[i] )
    if i in ( [ TARGET_F, TARGET_A ] ) : continue
    if dt[i] in (["object"]) : objList.append( i )


for i in objList :
    thePrefix = "z_" + i
    y = pd.get_dummies( df[i], prefix=thePrefix )   
    #y = pd.get_dummies( df[i], prefix=thePrefix )   
    df = pd.concat( [df, y], axis=1 )
    #df = df.drop( i, axis=1 )

 here is i ..... TARGET_BAD_FLAG  ..... and here is the type int64
 here is i ..... TARGET_LOSS_AMT  ..... and here is the type float64
 here is i ..... LOAN  ..... and here is the type int64
 here is i ..... MORTDUE  ..... and here is the type float64
 here is i ..... VALUE  ..... and here is the type float64
 here is i ..... YOJ  ..... and here is the type float64
 here is i ..... DEROG  ..... and here is the type float64
 here is i ..... DELINQ  ..... and here is the type float64
 here is i ..... CLAGE  ..... and here is the type float64
 here is i ..... NINQ  ..... and here is the type float64
 here is i ..... CLNO  ..... and here is the type float64
 here is i ..... DEBTINC  ..... and here is the type float64
 here is i ..... IMP_REASON  ..... and here is the type object
 here is i ..... IMP_JOB  ..... and here is the type object


In [8]:
g = df.groupby("IMP_JOB")
i = "MORTDUE"
print( g[i].median() )

IMP_JOB
MISSING    58936.0
Mgr        76600.0
Office     62970.5
Other      54834.0
ProfExe    82802.5
Sales      71800.0
Self       88465.0
Name: MORTDUE, dtype: float64


In [9]:
g = df.groupby("IMP_JOB")
i = "VALUE"
print( g[i].median() )

IMP_JOB
MISSING     78227.0
Mgr        101258.0
Office      89094.5
Other       76599.5
ProfExe    110007.0
Sales       84473.5
Self       130631.0
Name: VALUE, dtype: float64


In [10]:
g = df.groupby("IMP_REASON")
i = "NINQ"
print( g[i].median() )

IMP_REASON
DebtCon    1.0
HomeImp    0.0
MISSING    0.0
Name: NINQ, dtype: float64


In [11]:
i = "MORTDUE"
FLAG = "M_" + i
IMP = "IMP_" + i
#print( i )
#print( FLAG )
#print( IMP )
df[ FLAG ] = df[i].isna() + 0
df[ IMP ] = df[ i ]
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["MISSING"]), IMP ] = 58936
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Mgr"]), IMP ] = 76600
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Office"]), IMP ] = 62970
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Other"]), IMP ] = 54834
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["ProfExe"]), IMP ] = 82802
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Sales"]), IMP ] = 71800
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Self"]), IMP ] = 88465
df.loc[ df[IMP].isna(), IMP ] = df[i].median()
df = df.drop( i, axis=1 )
numList.remove(i)

i = "VALUE"
FLAG = "M_" + i
IMP = "IMP_" + i
#print( i )
#print( FLAG )
#print( IMP )
df[ FLAG ] = df[i].isna() + 0
df[ IMP ] = df[ i ]
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["MISSING"]), IMP ] = 78227
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Mgr"]), IMP ] = 101258
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Office"]), IMP ] = 89094
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Other"]), IMP ] = 76599
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["ProfExe"]), IMP ] = 110007
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Sales"]), IMP ] = 84473
df.loc[ df[IMP].isna() & df["IMP_JOB"].isin(["Self"]), IMP ] = 130631
df.loc[ df[IMP].isna(), IMP ] = df[i].median()
df = df.drop( i, axis=1 )
numList.remove(i)

i = "NINQ"
FLAG = "M_" + i
IMP = "IMP_" + i
#print( i )
#print( FLAG )
#print( IMP )
df[ FLAG ] = df[i].isna() + 0
df[ IMP ] = df[ i ]
df.loc[ df[IMP].isna() & df["IMP_REASON"].isin(["DebtCon"]), IMP ] = 1
df.loc[ df[IMP].isna() & df["IMP_REASON"].isin(["HomeImp"]), IMP ] = 0
df.loc[ df[IMP].isna() & df["IMP_REASON"].isin(["MISSING"]), IMP ] = 0

df.loc[ df[IMP].isna(), IMP ] = df[i].median()
df = df.drop( i, axis=1 )
numList.remove(i)



for i in numList :
    if df[i].isna().sum() == 0 : continue
    FLAG = "M_" + i
    IMP = "IMP_" + i
    #print(i)
    #print( df[i].isna().sum() )
    #print( FLAG )
    #print( IMP )
    #print(" ------- ")
    df[ FLAG ] = df[i].isna() + 0
    df[ IMP ] = df[ i ]
    df.loc[ df[IMP].isna(), IMP ] = df[i].median()
    df = df.drop( i, axis=1 )


for i in objList:
    df = df.drop( i, axis=1 )


df.head().T


,0,1,2,3,4
TARGET_BAD_FLAG,1.000000,1.000000,1.000000,1.000000,0.000000
TARGET_LOSS_AMT,641.000000,1109.000000,767.000000,1425.000000,NaN
LOAN,1100.000000,1300.000000,1500.000000,1500.000000,1700.000000
z_IMP_REASON_DebtCon,0.000000,0.000000,0.000000,0.000000,0.000000
z_IMP_REASON_HomeImp,1.000000,1.000000,1.000000,0.000000,1.000000
z_IMP_REASON_MISSING,0.000000,0.000000,0.000000,1.000000,0.000000
z_IMP_JOB_MISSING,0.000000,0.000000,0.000000,1.000000,0.000000
z_IMP_JOB_Mgr,0.000000,0.000000,0.000000,0.000000,0.000000
z_IMP_JOB_Office,0.000000,0.000000,0.000000,0.000000,1.000000
z_IMP_JOB_Other,1.000000,1.000000,1.000000,0.000000,0.000000


In [12]:
#Remove the Target Variables, Missing Flag fields and the Categorical Flag Fields

df_newTest = df.loc[:, ~df.columns.str.startswith('M_')]
df_newTest = df_newTest.loc[:, ~df_newTest.columns.str.startswith('z_')]
df_newTest = df_newTest.drop( ['TARGET_BAD_FLAG','TARGET_LOSS_AMT'], axis=1 )

df_newTest.describe().T

,count,mean,std,min,25%,50%,75%,max
LOAN,5960.0,18607.969799,11207.480417,1100.000000,11100.000000,16300.000000,23300.000000,89900.000000
IMP_MORTDUE,5960.0,72559.197013,42732.957883,2063.000000,48139.000000,62970.000000,88465.000000,399550.000000
IMP_VALUE,5960.0,101553.332054,56906.421205,8000.000000,66489.500000,89094.000000,119144.750000,855909.000000
IMP_NINQ,5960.0,1.127349,1.670567,0.000000,0.000000,1.000000,2.000000,17.000000
IMP_YOJ,5960.0,8.756166,7.259424,0.000000,3.000000,7.000000,12.000000,41.000000
IMP_DEROG,5960.0,0.224329,0.798458,0.000000,0.000000,0.000000,0.000000,10.000000
IMP_DELINQ,5960.0,0.405705,1.079256,0.000000,0.000000,0.000000,0.000000,15.000000
IMP_CLAGE,5960.0,179.440725,83.574697,0.000000,117.371430,173.466667,227.143058,1168.233561
IMP_CLNO,5960.0,21.247819,9.951308,0.000000,15.000000,20.000000,26.000000,71.000000
IMP_DEBTINC,5960.0,34.000651,7.644528,0.524499,30.763159,34.818262,37.949892,203.312149


In [13]:
z = np.abs(stats.zscore(df_newTest))
print(z)
print( "\n\n")

df_newTest_NoOutliers = df_newTest[(z < 3).all(axis=1)]

print("Records and Columns with Outliers")
print(df_newTest.shape)
print("Records and Columns without Outliers")
print(df_newTest_NoOutliers.shape)

[[1.5622993  1.09290621 1.0988844  ... 1.01802579 1.23087808 0.10696269]
 [1.54445258 0.05865279 0.58264275 ... 0.68935009 0.72838939 0.10696269]
 [1.52660585 1.38216859 1.49122805 ... 0.35868001 1.13038034 0.10696269]
 ...
 [6.29918148 0.4332897  0.15165347 ... 0.39296256 0.62789165 0.20355343]
 [6.35272165 0.51929611 0.17033482 ... 0.41226443 0.52739392 0.04451025]
 [6.36164501 0.55578155 0.22177446 ... 0.48057185 0.52739392 0.07468295]]



Records and Columns with Outliers
(5960, 10)
Records and Columns without Outliers
(5241, 10)


In [14]:
df_newTest_NoOutliers.describe().T

,count,mean,std,min,25%,50%,75%,max
LOAN,5241.0,17388.265598,8887.417647,1100.000000,10900.000000,15900.000000,22400.000000,52200.000000
IMP_MORTDUE,5241.0,69035.790952,34184.161035,2063.000000,48000.000000,62970.000000,86177.000000,200731.000000
IMP_VALUE,5241.0,96981.217142,44355.450515,8000.000000,66251.000000,88105.000000,117279.000000,271676.000000
IMP_NINQ,5241.0,0.947338,1.226260,0.000000,0.000000,1.000000,1.000000,6.000000
IMP_YOJ,5241.0,8.609760,6.992389,0.000000,3.000000,7.000000,12.000000,30.000000
IMP_DEROG,5241.0,0.120206,0.392747,0.000000,0.000000,0.000000,0.000000,2.000000
IMP_DELINQ,5241.0,0.253768,0.636439,0.000000,0.000000,0.000000,0.000000,3.000000
IMP_CLAGE,5241.0,177.224615,77.074246,0.507115,117.045501,173.466667,224.601376,427.923591
IMP_CLNO,5241.0,20.742606,9.152856,0.000000,15.000000,20.000000,26.000000,51.000000
IMP_DEBTINC,5241.0,33.863771,6.102162,11.652739,30.674205,34.818262,37.950788,55.962022


In [15]:
### MIN MAX SCALER
print(" NORMALIZING THE DATA \n\n\n")
theScaler = MinMaxScaler()
theScaler.fit( df_newTest_NoOutliers )

X_MINMAX = theScaler.transform( df_newTest_NoOutliers )
X_MINMAX = pd.DataFrame( X_MINMAX )

varNames = df_newTest_NoOutliers.columns
print( X_MINMAX.head() )
print( "\n\n")

 NORMALIZING THE DATA 



          0         1         2         3         4    5         6         7  \
0  0.000000  0.119783  0.117663  0.166667  0.350000  0.0  0.000000  0.219597   
1  0.003914  0.342229  0.229069  0.000000  0.233333  0.0  0.666667  0.283859   
2  0.007828  0.057568  0.032995  0.166667  0.133333  0.0  0.000000  0.348511   
3  0.007828  0.286272  0.266338  0.000000  0.233333  0.0  0.000000  0.404663   
4  0.011742  0.481894  0.394423  0.000000  0.100000  0.0  0.000000  0.217180   

          8         9  
0  0.176471  0.522814  
1  0.274510  0.522814  
2  0.196078  0.522814  
3  0.392157  0.522814  
4  0.274510  0.522814  





In [16]:
varNames_minmax = []
for i in varNames :
    newName = "nor_" + i
    varNames_minmax.append( newName )
print( varNames_minmax )
print( "\n\n")

['nor_LOAN', 'nor_IMP_MORTDUE', 'nor_IMP_VALUE', 'nor_IMP_NINQ', 'nor_IMP_YOJ', 'nor_IMP_DEROG', 'nor_IMP_DELINQ', 'nor_IMP_CLAGE', 'nor_IMP_CLNO', 'nor_IMP_DEBTINC']





In [17]:
X_MINMAX.columns = varNames_minmax
print( X_MINMAX.describe() )
print( "\n\n")

# This is where the dataset is good to use, variables have been scaled and you bring back in the Target and categorical variables
X_MINMAX[ "TARGET_BAD_FLAG" ] = df_org.TARGET_BAD_FLAG
X_MINMAX[ "TARGET_LOSS_AMT" ] = df_org.TARGET_LOSS_AMT
X_MINMAX[ "REASON" ] = df_org.REASON
X_MINMAX[ "JOB" ] = df_org.JOB
print( X_MINMAX.head() )
print( "\n\n")

          nor_LOAN  nor_IMP_MORTDUE  nor_IMP_VALUE  nor_IMP_NINQ  nor_IMP_YOJ  \
count  5241.000000      5241.000000    5241.000000   5241.000000  5241.000000   
mean      0.318753         0.337109       0.337464      0.157890     0.286992   
std       0.173922         0.172067       0.168220      0.204377     0.233080   
min       0.000000         0.000000       0.000000      0.000000     0.000000   
25%       0.191781         0.231225       0.220919      0.000000     0.100000   
50%       0.289628         0.306577       0.303801      0.166667     0.233333   
75%       0.416830         0.423390       0.414444      0.166667     0.400000   
max       1.000000         1.000000       1.000000      1.000000     1.000000   

       nor_IMP_DEROG  nor_IMP_DELINQ  nor_IMP_CLAGE  nor_IMP_CLNO  \
count    5241.000000     5241.000000    5241.000000   5241.000000   
mean        0.060103        0.084589       0.413455      0.406718   
std         0.196373        0.212146       0.180326      0.1794

In [18]:
#Fill Blank categorical values with 'MISSING'

dt2 = X_MINMAX.dtypes

objList2 = []

for i in dt2.index :
    #print(" here is i .....", i , " ..... and here is the type", dt[i] )
    if i in ( [ TARGET_F, TARGET_A ] ) : continue
    if dt2[i] in (["object"]) : objList2.append( i )

for i in objList2 :
    if X_MINMAX[i].isna().sum() == 0 : continue
    NAME = "IMP_"+i
    X_MINMAX[NAME] = X_MINMAX[i]
    X_MINMAX[NAME] = X_MINMAX[NAME].fillna("MISSING")
    g = X_MINMAX.groupby( NAME )
    X_MINMAX = X_MINMAX.drop( i, axis=1 )
X_MINMAX.head().T

,0,1,2,3,4
nor_LOAN,0,0.00391389,0.00782779,0.00782779,0.0117417
nor_IMP_MORTDUE,0.119783,0.342229,0.0575684,0.286272,0.481894
nor_IMP_VALUE,0.117663,0.229069,0.032995,0.266338,0.394423
nor_IMP_NINQ,0.166667,0,0.166667,0,0
nor_IMP_YOJ,0.35,0.233333,0.133333,0.233333,0.1
nor_IMP_DEROG,0,0,0,0,0
nor_IMP_DELINQ,0,0.666667,0,0,0
nor_IMP_CLAGE,0.219597,0.283859,0.348511,0.404663,0.21718
nor_IMP_CLNO,0.176471,0.27451,0.196078,0.392157,0.27451
nor_IMP_DEBTINC,0.522814,0.522814,0.522814,0.522814,0.522814


In [19]:
#This combines the original data minus the target variable with the normalized variables plus the target variable.
X_NEW = pd.concat([ df_newTest_NoOutliers , X_MINMAX ], axis=1 )
print( X_NEW.head() )
print( "\n\n")

     LOAN  IMP_MORTDUE  IMP_VALUE  IMP_NINQ  IMP_YOJ  IMP_DEROG  IMP_DELINQ  \
0  1100.0      25860.0    39025.0       1.0     10.5        0.0         0.0   
1  1300.0      70053.0    68400.0       0.0      7.0        0.0         2.0   
2  1500.0      13500.0    16700.0       1.0      4.0        0.0         0.0   
3  1500.0      58936.0    78227.0       0.0      7.0        0.0         0.0   
4  1700.0      97800.0   112000.0       0.0      3.0        0.0         0.0   

    IMP_CLAGE  IMP_CLNO  IMP_DEBTINC  nor_LOAN  nor_IMP_MORTDUE  \
0   94.366667       9.0    34.818262  0.000000         0.119783   
1  121.833333      14.0    34.818262  0.003914         0.342229   
2  149.466667      10.0    34.818262  0.007828         0.057568   
3  173.466667      20.0    34.818262  0.007828         0.286272   
4   93.333333      14.0    34.818262  0.011742         0.481894   

   nor_IMP_VALUE  nor_IMP_NINQ  nor_IMP_YOJ  nor_IMP_DEROG  nor_IMP_DELINQ  \
0       0.117663      0.166667     0.350000 

In [20]:
# Shows the manually calculated scaled values equals the MinMaxScaler values..
# Also shows if you have a max value of 10000 (outlier) that it will scale it near zero so make sure you handle outliers
# prior to scaling
X_TEST = X_NEW[ ["IMP_MORTDUE", "nor_IMP_MORTDUE" ] ]
TEMP = ( X_TEST["IMP_MORTDUE"] - 2063 ) / ( 200731 - 2063 )       # 4.3 is the original min value...7.9 is the original max value
X_TEST = X_TEST.assign( calc_IMP_MORTDUE = TEMP.values )
print( X_TEST.head() ) 
print( "\n\n")

   IMP_MORTDUE  nor_IMP_MORTDUE  calc_IMP_MORTDUE
0      25860.0         0.119783          0.119783
1      70053.0         0.342229          0.342229
2      13500.0         0.057568          0.057568
3      58936.0         0.286272          0.286272
4      97800.0         0.481894          0.481894





In [21]:
# STANDARD SCALER

print(" STANDARDIZING THE DATA \n\n\n")

theScaler = StandardScaler()
theScaler.fit( df_newTest_NoOutliers )

Y_STD = theScaler.transform( df_newTest_NoOutliers )
Y_STD = pd.DataFrame( Y_STD )
print( Y_STD.head() )
print( "\n\n")

 STANDARDIZING THE DATA 



          0         1         2         3         4         5         6  \
0 -1.832908 -1.263155 -1.306756  0.042949  0.270354 -0.306094 -0.398769   
1 -1.810402  0.029760 -0.644429 -0.772616 -0.230238 -0.306094  2.744014   
2 -1.787896 -1.624761 -1.810124  0.042949 -0.659317 -0.306094 -0.398769   
3 -1.787896 -0.295481 -0.422857 -0.772616 -0.230238 -0.306094 -0.398769   
4 -1.765390  0.841529  0.338633 -0.772616 -0.802343 -0.306094 -0.398769   

          7         8         9  
0 -1.075143 -1.283067  0.156433  
1 -0.718743 -0.736737  0.156433  
2 -0.360180 -1.173801  0.156433  
3 -0.048762 -0.081142  0.156433  
4 -1.088552 -0.736737  0.156433  





In [22]:
varNames_std = []
for i in varNames :
    newName = "std_" + i
    varNames_std.append( newName )

Y_STD.columns = varNames_std
print( Y_STD.head() )
print( "\n\n")

print( Y_STD.describe() )
print( "\n\n")

   std_LOAN  std_IMP_MORTDUE  std_IMP_VALUE  std_IMP_NINQ  std_IMP_YOJ  \
0 -1.832908        -1.263155      -1.306756      0.042949     0.270354   
1 -1.810402         0.029760      -0.644429     -0.772616    -0.230238   
2 -1.787896        -1.624761      -1.810124      0.042949    -0.659317   
3 -1.787896        -0.295481      -0.422857     -0.772616    -0.230238   
4 -1.765390         0.841529       0.338633     -0.772616    -0.802343   

   std_IMP_DEROG  std_IMP_DELINQ  std_IMP_CLAGE  std_IMP_CLNO  std_IMP_DEBTINC  
0      -0.306094       -0.398769      -1.075143     -1.283067         0.156433  
1      -0.306094        2.744014      -0.718743     -0.736737         0.156433  
2      -0.306094       -0.398769      -0.360180     -1.173801         0.156433  
3      -0.306094       -0.398769      -0.048762     -0.081142         0.156433  
4      -0.306094       -0.398769      -1.088552     -0.736737         0.156433  



           std_LOAN  std_IMP_MORTDUE  std_IMP_VALUE  std_IMP_NINQ 

In [23]:
Y_STD[ "TARGET_BAD_FLAG" ] = df_org.TARGET_BAD_FLAG
Y_STD[ "TARGET_LOSS_AMT" ] = df_org.TARGET_LOSS_AMT
Y_STD[ "REASON" ] = df_org.REASON
Y_STD[ "JOB" ] = df_org.JOB

print( Y_STD.head() )
print( "\n\n")

#Fill Blank categorical values with 'MISSING'

dt3 = Y_STD.dtypes

objList3 = []

for i in dt3.index :
    #print(" here is i .....", i , " ..... and here is the type", dt[i] )
    if i in ( [ TARGET_F, TARGET_A ] ) : continue
    if dt3[i] in (["object"]) : objList3.append( i )

for i in objList3 :
    if Y_STD[i].isna().sum() == 0 : continue
    NAME = "IMP_"+i
    Y_STD[NAME] = Y_STD[i]
    Y_STD[NAME] = Y_STD[NAME].fillna("MISSING")
    g = Y_STD.groupby( NAME )
    Y_STD = Y_STD.drop( i, axis=1 )
Y_STD.head().T


   std_LOAN  std_IMP_MORTDUE  std_IMP_VALUE  std_IMP_NINQ  std_IMP_YOJ  \
0 -1.832908        -1.263155      -1.306756      0.042949     0.270354   
1 -1.810402         0.029760      -0.644429     -0.772616    -0.230238   
2 -1.787896        -1.624761      -1.810124      0.042949    -0.659317   
3 -1.787896        -0.295481      -0.422857     -0.772616    -0.230238   
4 -1.765390         0.841529       0.338633     -0.772616    -0.802343   

   std_IMP_DEROG  std_IMP_DELINQ  std_IMP_CLAGE  std_IMP_CLNO  \
0      -0.306094       -0.398769      -1.075143     -1.283067   
1      -0.306094        2.744014      -0.718743     -0.736737   
2      -0.306094       -0.398769      -0.360180     -1.173801   
3      -0.306094       -0.398769      -0.048762     -0.081142   
4      -0.306094       -0.398769      -1.088552     -0.736737   

   std_IMP_DEBTINC  TARGET_BAD_FLAG  TARGET_LOSS_AMT   REASON     JOB  
0         0.156433                1            641.0  HomeImp   Other  
1         0.156433  

,0,1,2,3,4
std_LOAN,-1.83291,-1.8104,-1.7879,-1.7879,-1.76539
std_IMP_MORTDUE,-1.26316,0.0297596,-1.62476,-0.295481,0.841529
std_IMP_VALUE,-1.30676,-0.644429,-1.81012,-0.422857,0.338633
std_IMP_NINQ,0.0429491,-0.772616,0.0429491,-0.772616,-0.772616
std_IMP_YOJ,0.270354,-0.230238,-0.659317,-0.230238,-0.802343
std_IMP_DEROG,-0.306094,-0.306094,-0.306094,-0.306094,-0.306094
std_IMP_DELINQ,-0.398769,2.74401,-0.398769,-0.398769,-0.398769
std_IMP_CLAGE,-1.07514,-0.718743,-0.36018,-0.0487622,-1.08855
std_IMP_CLNO,-1.28307,-0.736737,-1.1738,-0.0811416,-0.736737
std_IMP_DEBTINC,0.156433,0.156433,0.156433,0.156433,0.156433


In [24]:
Y_NEW = pd.concat([ df_newTest_NoOutliers , Y_STD ], axis=1 )
print(Y_NEW.head().T )
print( "\n\n")

                         0          1          2          3         4
LOAN                  1100       1300       1500       1500      1700
IMP_MORTDUE          25860      70053      13500      58936     97800
IMP_VALUE            39025      68400      16700      78227    112000
IMP_NINQ                 1          0          1          0         0
IMP_YOJ               10.5          7          4          7         3
IMP_DEROG                0          0          0          0         0
IMP_DELINQ               0          2          0          0         0
IMP_CLAGE          94.3667    121.833    149.467    173.467   93.3333
IMP_CLNO                 9         14         10         20        14
IMP_DEBTINC        34.8183    34.8183    34.8183    34.8183   34.8183
std_LOAN          -1.83291    -1.8104    -1.7879    -1.7879  -1.76539
std_IMP_MORTDUE   -1.26316  0.0297596   -1.62476  -0.295481  0.841529
std_IMP_VALUE     -1.30676  -0.644429   -1.81012  -0.422857  0.338633
std_IMP_NINQ     0.0

In [25]:
Y_TEST = Y_NEW[ ["IMP_MORTDUE", "std_IMP_MORTDUE" ] ]
TEMP = ( Y_TEST["IMP_MORTDUE"] - 69035.790952 ) / 34184.161035   # subtract the oiginal mean and divide by the original sd 
Y_TEST = Y_TEST.assign( calc_IMP_MORTDUE = TEMP.values )
print( Y_TEST.head() ) 
print( "\n\n")

   IMP_MORTDUE  std_IMP_MORTDUE  calc_IMP_MORTDUE
0      25860.0        -1.263155         -1.263035
1      70053.0         0.029760          0.029757
2      13500.0        -1.624761         -1.624606
3      58936.0        -0.295481         -0.295452
4      97800.0         0.841529          0.841448



